In [1]:
from __future__ import print_function
%matplotlib inline 

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import torch
import os
import sys
import glob
import h5py
import numpy as np
import math
import torch
from torch import nn
from torch.autograd import Variable
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.data import Dataset , DataLoader
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
from tqdm import tqdm
import tensorboard
import tensorboardX
from torch.utils.tensorboard import SummaryWriter
from log import Logger
from data import trainlabelDataset_reduced, testlabelDataset_reduced
from util import r2, mse, rmse, mae, pp_mse, pp_rmse, pp_mae
from model import  autoencoder_999, autoencoder_333, autoencoder_333_2, autoencoder_1014,autoencoder_1015
from sklearn.decomposition import PCA
import random 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



/n/home01/jyin/.conda/envs/py36/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/n/home01/jyin/.conda/envs/py36/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/n/home01/jyin/.conda/envs/py36/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/n/home01/jyin/.conda/envs/py36/li

In [118]:


a=np.random.rand(2,4,3)  # (N, C, H, W)
a=torch.from_numpy(a.astype('float32')).cuda()
b = nn.BatchNorm1d(4).cuda()(a)


In [119]:
a  # 2 * 4 * 3 

tensor([[[0.7718, 0.5505, 0.4935],
         [0.8771, 0.4735, 0.1719],
         [0.8126, 0.4825, 0.3412],
         [0.3548, 0.2136, 0.4274]],

        [[0.6164, 0.6061, 0.8300],
         [0.4874, 0.2090, 0.6246],
         [0.1444, 0.2041, 0.6101],
         [0.6729, 0.9977, 0.2313]]], device='cuda:0')

In [120]:
b  # 2 * 4 * 3 

tensor([[[ 1.0705e+00, -7.9374e-01, -1.2737e+00],
         [ 1.6769e+00, -1.8435e-03, -1.2559e+00],
         [ 1.6407e+00,  2.1595e-01, -3.9390e-01],
         [-4.6449e-01, -9.7652e-01, -2.0155e-01]],

        [[-2.3858e-01, -3.2489e-01,  1.5604e+00],
         [ 5.5893e-02, -1.1017e+00,  6.2668e-01],
         [-1.2433e+00, -9.8591e-01,  7.6646e-01],
         [ 6.8855e-01,  1.8663e+00, -9.1226e-01]]], device='cuda:0',
       grad_fn=<CudnnBatchNormBackward>)

In [137]:
np.mean([0.7718, 0.6164] )

0.6940999999999999

In [138]:
np.var([0.7718, 0.6164 ])

0.006037290000000008

In [143]:
(0.7718 -0.6940999999)/np.sqrt(0.00603729)

1.0000000012870012

In [139]:
np.mean([0.5505, 0.6061] )

0.5783

In [140]:
np.var([0.5505, 0.6061] )

0.0007728399999999995

In [144]:
(0.5505 -0.5783)/np.sqrt(0.0007728399)

-1.0000000646964524

In [141]:
np.mean([0.4935,0.8300])

0.66175

In [142]:
np.var([0.4935,0.8300])

0.028308062499999995

In [145]:
(0.4935 -0.66175)/np.sqrt(0.02830806)

-1.000000044157034

In [146]:
np.mean([1.0000000012870012,-1.0000000646964524, -1.000000044157034] )

-0.33333336918882844

In [147]:
np.var([1.0000000012870012,-1.0000000646964524, -1.000000044157034] )

0.8888889384122179

In [148]:
(1.0000000012870012- (-0.33333336918882844))/np.sqrt(0.8888889384122179)

1.4142135623730951

# here here here 

# basically what batch norm do is: 
  -1- eg (N,C,H) = (2,4,3) # batch size = 2, num of channels = 4 , each channel has 3 neurons 
  -2- for one chanel: batch norm mean = np.mean ( 6 neuron values [that from 2 datasets * 3 neurons each data set in this chanle]) 
  batch norm var = np.var ( 6 neuron values [that from 2 datasets * 3 neurons each data set in this chanle]) 
  
  -3- basically batch norm mean is the mean cross all neurons in one layer cross all datasets 

In [122]:
np.mean([0.7718, 0.5505, 0.4935,0.6164, 0.6061, 0.8300])


0.6447166666666667

In [123]:
np.var([0.7718, 0.5505, 0.4935,0.6164, 0.6061, 0.8300])


0.01408607138888889

In [132]:
np.var([0.7718, 0.5505, 0.4935])


0.01440817555555556

In [134]:
np.var([0.6164, 0.6061, 0.8300])


0.010651362222222223

In [125]:
(a[0][0][0] -0.6447166666666667 )/ np.sqrt(0.01408607138888889)

tensor(1.0708, device='cuda:0')

In [131]:
(a[0][0][1] -0.6447166666666667 )/ np.sqrt(0.01408607138888889)

tensor(-0.7940, device='cuda:0')

In [126]:
(a[1][0][0] -0.6447166666666667 )/ np.sqrt(0.01408607138888889)

tensor(-0.2386, device='cuda:0')